In [6]:
import pandas as pd
import re

def load_signal_data(file_path):
    """
    Parses the signal strength data from a text file into a DataFrame.
    """
    data = []
    
    with open(file_path, 'r') as file:
        for line in file:
            # Use regex to capture position and signal strength
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"router_position": (x, y), "signal_strength": signal_strength})
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print("Parsed data:\n", df.head())  # Debugging line
    return df

def calculate_coverage_score(data, min_signal=-70, max_signal=-50):
    """
    Calculates the coverage score for each position based on signal strength.
    """
    scores = {}
    for position, signals in data.groupby('router_position'):
        score = sum((min_signal <= strength <= max_signal) for strength in signals['signal_strength'])
        scores[position] = score
    return scores

def greedy_router_placement(data, coverage_goal=80):
    """
    Greedy algorithm to find the optimal router placement positions to maximize coverage.
    """
    scores = calculate_coverage_score(data)
    selected_positions = []
    covered_points = set()
    total_points = len(data['router_position'].unique())
    
    while (len(covered_points) / total_points) * 100 < coverage_goal:
        # Find the position with the highest remaining coverage score
        position = max(scores, key=scores.get)
        max_score = scores[position]
        
        # If no position can contribute further, stop
        if max_score == 0:
            print("Unable to reach the coverage goal with available positions.")
            break
        
        # Add the position to selected and remove it from scores to avoid reselection
        selected_positions.append(position)
        del scores[position]
        
        # Update covered points with this router's range
        coverage = data[(data['router_position'] == position) & 
                        (data['signal_strength'] >= -70) & 
                        (data['signal_strength'] <= -50)]
        for _, row in coverage.iterrows():
            covered_points.add(row['router_position'])
    
    return selected_positions, (len(covered_points) / total_points) * 100

def main():
    data = load_signal_data('signal_strength_by_router_position.txt')
    
    if data.empty:
        print("Error: No data was parsed from the file.")
        return
    
    coverage_goal = 80  # Adjustable coverage goal in percentage
    selected_positions, achieved_coverage = greedy_router_placement(data, coverage_goal)
    
    print(f"Selected Router Positions for Optimal Coverage: {selected_positions}")
    print(f"Achieved Coverage: {achieved_coverage}%")

if __name__ == "__main__":
    main()


Parsed data:
   router_position  signal_strength
0          (0, 0)      -122.478065
1          (0, 1)      -155.296433
2          (0, 2)      -156.807336
3          (0, 3)      -156.281831
4          (0, 4)      -158.810419
Unable to reach the coverage goal with available positions.
Selected Router Positions for Optimal Coverage: [(2, 3), (2, 4), (3, 5), (4, 6), (5, 6), (6, 7), (7, 8)]
Achieved Coverage: 7.000000000000001%


In [1]:
import pandas as pd
import re
import numpy as np
from scipy.spatial.distance import cdist

# Load signal data
def load_signal_data(file_path):
    """
    Parses the signal strength data from a text file into a DataFrame.
    """
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Use regex to capture position and signal strength
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"point": (x, y), "signal_strength": signal_strength})
    
    df = pd.DataFrame(data)
    return df

# Calculate coverage score
def calculate_coverage(data, router_positions, min_signal=-70, max_signal=-50):
    """
    Calculates coverage for given router positions.
    """
    coverage_count = 0
    covered_points = set()
    
    for position in router_positions:
        distances = cdist([position], data['point'].tolist(), 'euclidean').flatten()
        for idx, distance in enumerate(distances):
            if data['signal_strength'].iloc[idx] >= min_signal and data['signal_strength'].iloc[idx] <= max_signal:
                covered_points.add(data['point'].iloc[idx])
                
    return len(covered_points)

# Particle Swarm Optimization for router placement
class Particle:
    def __init__(self, bounds, num_routers):
        self.position = np.random.uniform(bounds[0], bounds[1], (num_routers, 2))
        self.velocity = np.random.uniform(-1, 1, (num_routers, 2))
        self.best_position = self.position.copy()
        self.best_score = float('-inf')

def pso(data, bounds, num_routers, num_particles=30, max_iter=10, min_signal=-70, max_signal=-50):
    """
    PSO to find optimal router placement for maximizing coverage.
    """
    particles = [Particle(bounds, num_routers) for _ in range(num_particles)]
    global_best_position = None
    global_best_score = float('-inf')
    
    w, c1, c2 = 0.5, 1.5, 1.5  # PSO parameters: inertia, cognitive, and social components
    
    for iteration in range(max_iter):
        for particle in particles:
            # Calculate coverage for each particle's position
            coverage_score = calculate_coverage(data, particle.position, min_signal, max_signal)
            
            # Update personal best
            if coverage_score > particle.best_score:
                particle.best_score = coverage_score
                particle.best_position = particle.position.copy()
            
            # Update global best
            if coverage_score > global_best_score:
                global_best_score = coverage_score
                global_best_position = particle.position.copy()
        
        # Update velocity and position
        for particle in particles:
            cognitive_component = c1 * np.random.rand() * (particle.best_position - particle.position)
            social_component = c2 * np.random.rand() * (global_best_position - particle.position)
            particle.velocity = w * particle.velocity + cognitive_component + social_component
            particle.position = np.clip(particle.position + particle.velocity, bounds[0], bounds[1])
        
        print(f"Iteration {iteration + 1}/{max_iter}, Best Coverage Score: {global_best_score}")
    
    return global_best_position, global_best_score

def main():
    # Load the signal data
    data = load_signal_data('signal_strength_by_router_position.txt')
    if data.empty:
        print("Error: No data was parsed from the file.")
        return

    # Define PSO parameters
    bounds = [(0, 20), (0, 20)]  # Adjust based on the layout dimensions
    num_routers = 3  # Adjust the number of routers based on requirements
    optimal_positions, best_score = pso(data, bounds, num_routers)

    print("Optimal Router Positions:", optimal_positions)
    print("Maximum Coverage Achieved:", best_score)

if __name__ == "__main__":
    main()


Iteration 1/10, Best Coverage Score: 7
Iteration 2/10, Best Coverage Score: 7
Iteration 3/10, Best Coverage Score: 7
Iteration 4/10, Best Coverage Score: 7
Iteration 5/10, Best Coverage Score: 7
Iteration 6/10, Best Coverage Score: 7
Iteration 7/10, Best Coverage Score: 7
Iteration 8/10, Best Coverage Score: 7
Iteration 9/10, Best Coverage Score: 7
Iteration 10/10, Best Coverage Score: 7
Optimal Router Positions: [[ 0. 20.]
 [ 0. 20.]
 [ 0. 20.]]
Maximum Coverage Achieved: 7


In [3]:
import pandas as pd
import random
import re
import os
# Load signal strength data
def load_signal_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"position": (x, y), "signal_strength": signal_strength})
    return pd.DataFrame(data)

# Initialize population
def initialize_population(data, population_size, num_routers):
    low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
    population = []
    for _ in range(population_size):
        chromosome = random.sample(low_signal_positions, num_routers)
        population.append(chromosome)
    return population

# Fitness function with debugging output
def fitness_function(chromosome, data, min_signal=-70, max_signal=-50, coverage_threshold=0.75):
    covered_points = 0
    total_points = len(data)
    
    for position in chromosome:
        signals = data[data['position'] == position]['signal_strength']
        if not signals.empty:
            signal_strength = signals.values[0]
            if min_signal <= signal_strength <= max_signal:
                covered_points += 1

    coverage = covered_points / total_points
    score = coverage * 100  # Convert to percentage
    
    penalty = len(chromosome) / 10
    final_score = score - penalty
    
    
    return final_score if coverage >= coverage_threshold else final_score - 100

# Selection (tournament selection)
def selection(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover with check for single-element chromosomes
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Skip crossover if not enough points
    
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation (randomly swap router positions)
def mutation(chromosome, low_signal_positions, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(low_signal_positions)
    return chromosome

# Genetic algorithm function with debugging
def genetic_algorithm(data, population_size=50, initial_routers=1, max_routers=10, generations=100, mutation_rate=0.1, coverage_threshold=0.75):
    num_routers = initial_routers
    best_placement = None
    best_score = -float('inf')
    
    while num_routers <= max_routers:
        population = initialize_population(data, population_size, num_routers)
        
        for generation in range(generations):
           
            fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
            population = selection(population, fitness_scores)
            
            next_generation = []
            for i in range(0, len(population), 2):
                parent1, parent2 = population[i], population[(i+1) % len(population)]
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            
            low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
            next_generation = [mutation(child, low_signal_positions, mutation_rate) for child in next_generation]
            
            population = next_generation
        
        # Evaluate final population fitness
        fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
        current_best_score = max(fitness_scores)
        current_best_chromosome = population[fitness_scores.index(current_best_score)]
        
        if current_best_score > best_score:
            best_score = current_best_score
            best_placement = current_best_chromosome
            
            if fitness_function(best_placement, data, coverage_threshold=coverage_threshold) >= coverage_threshold * 100:
                break
            
        num_routers += 1

    return best_placement, best_score

def main():
    # Remove old report.txt if it exists
    report_file_path = 'report.txt'
    if os.path.exists(report_file_path):
        os.remove(report_file_path)

    data = load_signal_data('signal_strength_by_router_position.txt')
    
    population_size = 50
    initial_routers = 1
    max_routers = 4
    generations = 100
    mutation_rate = 0.1
    coverage_threshold = 0.75  # Adjusted threshold for debugging
    
    best_placement, best_score = genetic_algorithm(
        data, 
        population_size, 
        initial_routers, 
        max_routers, 
        generations, 
        mutation_rate, 
        coverage_threshold
    )
    
    # Format the output as specified
    num_routers = len(best_placement)
    report_content = f"Optimal Router Positions: ({num_routers}, {best_placement})\n"
    

    # Display in console
    print(report_content)
    
    # Save results to report.txt in specified format
    with open(report_file_path, 'w') as report_file:
        report_file.write(report_content)

if __name__ == "__main__":
    main()

Optimal Router Positions: (1, [(5, 7)])
Coverage Score at Optimal Placement: -100.1


In [11]:
import pandas as pd
import random
import re
import os

# Load signal strength data
def load_signal_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"position": (x, y), "signal_strength": signal_strength})
    return pd.DataFrame(data)

# Load wall and obstacle data from combined_data.txt
def load_combined_data(file_path):
    walls = []
    obstacles = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # Get dimensions (length and width) if needed
        length, width = map(float, lines[0:2])  # Length and width, if needed for other calculations
        
        # Number of obstacles
        num_obstacles = int(lines[2])
        
        # Read wall and obstacle data
        for line in lines[3:3 + num_obstacles]:  # Read obstacles
            parts = line.split()
            if len(parts) >= 3:
                x = float(parts[0])
                y = float(parts[1])
                material = parts[2]
                obstacles.append({"position": (x, y), "material": material})
        
        for line in lines[3 + num_obstacles:]:  # Read walls
            parts = line.split()
            if len(parts) >= 4:
                start_x = float(parts[0])
                start_y = float(parts[1])
                end_x = float(parts[2])
                end_y = float(parts[3])
                material = parts[4]
                walls.append({"start": (start_x, start_y), "end": (end_x, end_y), "material": material})

    return obstacles, walls

# Filter low signal positions away from obstacles
def filter_positions(data, obstacles, min_distance=1.5):
    obstacle_positions = [(obs['position'][0], obs['position'][1]) for obs in obstacles]
    allowed_positions = []
    for index, row in data.iterrows():
        pos = row['position']
        if all(abs(pos[0] - obs_pos[0]) > min_distance or abs(pos[1] - obs_pos[1]) > min_distance for obs_pos in obstacle_positions):
            allowed_positions.append(pos)
    return allowed_positions

# Initialize population
def initialize_population(filtered_positions, population_size, num_routers):
    population = []
    for _ in range(population_size):
        chromosome = random.sample(filtered_positions, num_routers)
        population.append(chromosome)
    return population

# Fitness function with penalties for walls and obstacles
def fitness_function(chromosome, data, obstacles, walls, min_signal=-70, max_signal=-50, coverage_threshold=0.75):
    covered_points = 0
    total_points = len(data)
    
    for position in chromosome:
        signals = data[data['position'] == position]['signal_strength']
        if not signals.empty:
            signal_strength = signals.values[0]
            if min_signal <= signal_strength <= max_signal and position not in [wall['start'] for wall in walls] and position not in [wall['end'] for wall in walls] and position not in [obs['position'] for obs in obstacles]:
                covered_points += 1

    coverage = covered_points / total_points
    score = coverage * 100  # Convert to percentage
    
    penalty = len(chromosome) / 10
    final_score = score - penalty
    
    return final_score if coverage >= coverage_threshold else final_score - 100

# Selection (tournament selection)
def selection(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover with check for single-element chromosomes
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Skip crossover if not enough points
    
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation (randomly swap router positions), avoiding walls and obstacles
def mutation(chromosome, low_signal_positions, obstacles, walls, mutation_rate=0.1):
    valid_positions = [pos for pos in low_signal_positions if pos not in [obs['position'] for obs in obstacles] and pos not in [wall['start'] for wall in walls] and pos not in [wall['end'] for wall in walls]]
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(valid_positions)
    return chromosome 

# Genetic algorithm function with combined data consideration
def genetic_algorithm(data, obstacles, walls, population_size=50, initial_routers=1, max_routers=10, generations=100, mutation_rate=0.1, coverage_threshold=0.75):
    num_routers = initial_routers
    best_placement = None
    best_score = -float('inf')
    
    # Filter positions based on signal strength and obstacles
    filtered_positions = filter_positions(data, obstacles)

    while num_routers <= max_routers:
        # Initialize population with filtered positions
        population = initialize_population(filtered_positions, population_size, num_routers)
        
        for generation in range(generations):
            # Calculate fitness scores for the current population
            fitness_scores = [fitness_function(chromosome, data, obstacles, walls, coverage_threshold=coverage_threshold) for chromosome in population]
            population = selection(population, fitness_scores)
            
            next_generation = []
            for i in range(0, len(population), 2):
                parent1, parent2 = population[i], population[(i + 1) % len(population)]
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            
            # Get low signal positions for mutation
            low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
            next_generation = [mutation(child, low_signal_positions, obstacles, walls, mutation_rate) for child in next_generation]
            
            population = next_generation
        
        # Evaluate the fitness of the final population
        fitness_scores = [fitness_function(chromosome, data, obstacles, walls, coverage_threshold=coverage_threshold) for chromosome in population]
        current_best_score = max(fitness_scores)
        current_best_chromosome = population[fitness_scores.index(current_best_score)]
        
        if current_best_score > best_score:
            best_score = current_best_score
            best_placement = current_best_chromosome
            
            # Break if the coverage is satisfactory
            if fitness_function(best_placement, data, obstacles, walls, coverage_threshold=coverage_threshold) >= coverage_threshold * 100:
                break
            
        num_routers += 1

    return best_placement

def main():
    # Remove old report.txt if it exists
    report_file_path = 'report.txt'
    if os.path.exists(report_file_path):
        os.remove(report_file_path)

    # Load signal data
    data = load_signal_data('signal_degradation.txt')

    # Load wall and obstacle data
    obstacles, walls = load_combined_data('combined_data.txt')

    # Genetic algorithm parameters
    population_size = 50
    initial_routers = 1
    max_routers = 4
    generations = 100  # Make sure this is an integer
    mutation_rate = 0.1
    coverage_threshold = 0.75  # Adjusted threshold for debugging

    # Run the genetic algorithm
    best_placement = genetic_algorithm(
        data,
        obstacles,
        walls,
        population_size,
        initial_routers,
        max_routers,
        generations,
        mutation_rate,
        coverage_threshold
    )

    # Format and save results
    num_routers = len(best_placement)
    report_content = f"Optimal Router Positions: ({num_routers}, {best_placement})\n"

    # Display in console
    print(report_content)

    # Save results to report.txt in specified format
    with open(report_file_path, 'w') as report_file:
        report_file.write(report_content)

if __name__ == "__main__":
    main()


Optimal Router Positions: (1, [(4, 1)])



In [1]:
import pandas as pd
import random
import re
import os
import time 

def calculate_execution_time():
    start_time = time.time()  

# Load signal strength data
def load_signal_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"position": (x, y), "signal_strength": signal_strength})
    return pd.DataFrame(data)

# Initialize population with positions close to the layout center
def initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center=10):
    low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
    centralized_positions = [
        pos for pos in low_signal_positions 
        if abs(pos[0] - layout_center[0]) <= max_distance_from_center 
        and abs(pos[1] - layout_center[1]) <= max_distance_from_center
    ]

    population = []
    for _ in range(population_size):
        chromosome = random.sample(centralized_positions, num_routers)
        population.append(chromosome)
    return population

# Fitness function with debugging output
def fitness_function(chromosome, data, min_signal=-70, max_signal=-50, coverage_threshold=0.75):
    covered_points = 0
    total_points = len(data)
    
    for position in chromosome:
        signals = data[data['position'] == position]['signal_strength']
        if not signals.empty:
            signal_strength = signals.values[0]
            if min_signal <= signal_strength <= max_signal:
                covered_points += 1

    coverage = covered_points / total_points
    score = coverage * 100  # Convert to percentage
    
    penalty = len(chromosome) / 10
    final_score = score - penalty
    
    return final_score if coverage >= coverage_threshold else final_score - 100

# Selection (tournament selection)
def selection(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover with check for single-element chromosomes
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Skip crossover if not enough points
    
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation (randomly swap router positions)
def mutation(chromosome, low_signal_positions, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(low_signal_positions)
    return chromosome

# Genetic algorithm function with debugging
def genetic_algorithm(data, population_size=50, initial_routers=1, max_routers=10, generations=100, mutation_rate=0.1, coverage_threshold=0.75, layout_center=(0, 0), max_distance_from_center=10):
    num_routers = initial_routers
    best_placement = None
    best_score = -float('inf')
    
    while num_routers <= max_routers:
        population = initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center)
        
        for generation in range(generations):
            fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
            population = selection(population, fitness_scores)
            
            next_generation = []
            for i in range(0, len(population), 2):
                parent1, parent2 = population[i], population[(i+1) % len(population)]
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            
            low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
            next_generation = [mutation(child, low_signal_positions, mutation_rate) for child in next_generation]
            
            population = next_generation
        
        # Evaluate final population fitness
        fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
        current_best_score = max(fitness_scores)
        current_best_chromosome = population[fitness_scores.index(current_best_score)]
        
        if current_best_score > best_score:
            best_score = current_best_score
            best_placement = current_best_chromosome
            
            if fitness_function(best_placement, data, coverage_threshold=coverage_threshold) >= coverage_threshold * 100:
                break
            
        num_routers += 1

    return best_placement

def main():
  
   
    data = load_signal_data('signal_degradation.txt')
    
    # Calculate layout center based on signal data
    layout_center_x = (data['position'].apply(lambda p: p[0]).max() + data['position'].apply(lambda p: p[0]).min()) // 2
    layout_center_y = (data['position'].apply(lambda p: p[1]).max() + data['position'].apply(lambda p: p[1]).min()) // 2
    layout_center = (layout_center_x, layout_center_y)

    # Genetic algorithm parameters
    population_size = 50
    initial_routers = 1
    max_routers = 4
    generations = 100
    mutation_rate = 0.1
    coverage_threshold = 0.75
    max_distance_from_center = 15  # Adjust as needed

    best_placement = genetic_algorithm(
        data, 
        population_size, 
        initial_routers, 
        max_routers, 
        generations, 
        mutation_rate, 
        coverage_threshold,
        layout_center,
        max_distance_from_center
    )
    
    # Format the output as specified
    num_routers = len(best_placement)
    report_content = f"Optimal Router Positions: ({num_routers}, {best_placement})\n"
    
    # Display in console
    print(report_content)
    
    # Save results to report.txt in specified format
    report_file_path = 'report.txt' 
    with open(report_file_path, 'w') as report_file:
        report_file.write(report_content)

if __name__ == "__main__":
    main()


Optimal Router Positions: (1, [(12, 6)])



In [2]:
import time

# Define a function to calculate execution time of the genetic algorithm
def calculate_execution_time():
    start_time = time.time()  # Record the start time
    
    
    
    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    
    # Print the execution time
    print(f"Execution time: {execution_time:.4f} seconds")

# Run the execution time function
calculate_execution_time()


Execution time: 0.0000 seconds


In [4]:
import pandas as pd
import random
import re
import os
import time  # Importing time module

# Load signal strength data
def load_signal_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
            if match:
                x = int(match.group(1))
                y = int(match.group(2))
                signal_strength = float(match.group(3))
                data.append({"position": (x, y), "signal_strength": signal_strength})
    return pd.DataFrame(data)

# Initialize population with positions close to the layout center
def initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center=10):
    low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
    centralized_positions = [
        pos for pos in low_signal_positions 
        if abs(pos[0] - layout_center[0]) <= max_distance_from_center 
        and abs(pos[1] - layout_center[1]) <= max_distance_from_center
    ]

    population = []
    for _ in range(population_size):
        chromosome = random.sample(centralized_positions, num_routers)
        population.append(chromosome)
    return population

# Fitness function with debugging output
def fitness_function(chromosome, data, min_signal=-70, max_signal=-50, coverage_threshold=0.75):
    covered_points = 0
    total_points = len(data)
    
    for position in chromosome:
        signals = data[data['position'] == position]['signal_strength']
        if not signals.empty:
            signal_strength = signals.values[0]
            if min_signal <= signal_strength <= max_signal:
                covered_points += 1

    coverage = covered_points / total_points
    score = coverage * 100  # Convert to percentage
    
    penalty = len(chromosome) / 10
    final_score = score - penalty
    
    return final_score if coverage >= coverage_threshold else final_score - 100

# Selection (tournament selection)
def selection(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover with check for single-element chromosomes
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Skip crossover if not enough points
    
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation (randomly swap router positions)
def mutation(chromosome, low_signal_positions, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(low_signal_positions)
    return chromosome

# Genetic algorithm function with debugging
def genetic_algorithm(data, population_size=50, initial_routers=1, max_routers=10, generations=100, mutation_rate=0.1, coverage_threshold=0.75, layout_center=(0, 0), max_distance_from_center=10):
    num_routers = initial_routers
    best_placement = None
    best_score = -float('inf')
    
    while num_routers <= max_routers:
        population = initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center)
        
        for generation in range(generations):
            fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
            population = selection(population, fitness_scores)
            
            next_generation = []
            for i in range(0, len(population), 2):
                parent1, parent2 = population[i], population[(i+1) % len(population)]
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            
            low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
            next_generation = [mutation(child, low_signal_positions, mutation_rate) for child in next_generation]
            
            population = next_generation
        
        # Evaluate final population fitness
        fitness_scores = [fitness_function(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
        current_best_score = max(fitness_scores)
        current_best_chromosome = population[fitness_scores.index(current_best_score)]
        
        if current_best_score > best_score:
            best_score = current_best_score
            best_placement = current_best_chromosome
            
            if fitness_function(best_placement, data, coverage_threshold=coverage_threshold) >= coverage_threshold * 100:
                break
            
        num_routers += 1

    return best_placement

def main():
    # Record start time
    start_time = time.time()

    data = load_signal_data('signal_degradation.txt')
    
    # Calculate layout center based on signal data
    layout_center_x = (data['position'].apply(lambda p: p[0]).max() + data['position'].apply(lambda p: p[0]).min()) // 2
    layout_center_y = (data['position'].apply(lambda p: p[1]).max() + data['position'].apply(lambda p: p[1]).min()) // 2
    layout_center = (layout_center_x, layout_center_y)

    # Genetic algorithm parameters
    population_size = 50
    initial_routers = 1
    max_routers = 4
    generations = 100
    mutation_rate = 0.1
    coverage_threshold = 0.75
    max_distance_from_center = 15  # Adjust as needed

    best_placement = genetic_algorithm(
        data, 
        population_size, 
        initial_routers, 
        max_routers, 
        generations, 
        mutation_rate, 
        coverage_threshold,
        layout_center,
        max_distance_from_center
    )
    
    # Format the output as specified
    num_routers = len(best_placement)
    report_content = f"Optimal Router Positions: ({num_routers}, {best_placement})\n"
    
    # Display in console
    print(report_content)
    
    # Save results to report.txt in specified format
    report_file_path = 'report.txt' 
    with open(report_file_path, 'w') as report_file:
        report_file.write(report_content)

    # Record end time and calculate execution duration
    end_time = time.time()
    execution_time = end_time - start_time

    # Display the execution time
    print(f"Execution Time: {execution_time:.2f} seconds")

if __name__ == "__main__":
    main()


Optimal Router Positions: (1, [(1, 7)])

Execution Time: 112.00 seconds


In [1]:
import pandas as pd
import random
import re
import os

# Load signal strength data
def load_signal_data(file_path):
    data = []
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            for line in file:
                match = re.match(r'Point \((\d+),\s*(\d+)\): Received Power = ([-\d.]+) dBm', line)
                if match:
                    x = int(match.group(1))
                    y = int(match.group(2))
                    signal_strength = float(match.group(3))
                    data.append({"position": (x, y), "signal_strength": signal_strength})
    else:
        print(f"File {file_path} not found.")
    return pd.DataFrame(data)

# Initialize population with positions close to the layout center
def initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center=10):
    low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
    centralized_positions = [
        pos for pos in low_signal_positions 
        if abs(pos[0] - layout_center[0]) <= max_distance_from_center 
        and abs(pos[1] - layout_center[1]) <= max_distance_from_center
    ]

    population = []
    for _ in range(population_size):
        chromosome = random.sample(centralized_positions, min(num_routers, len(centralized_positions)))
        population.append(chromosome)
    return population

# Fitness function with higher mutation rate testing

def fitness_function_high_mutation(chromosome, data, min_signal=-70, max_signal=-50, coverage_threshold=0.75):
    """
    Calculates the fitness score for a chromosome, evaluating the percentage of points covered 
    within the desired signal strength range, while also encouraging fewer routers if possible.
    """
    # Initialize counters
    covered_points = 0
    total_points = len(data)
    
    # Check each router position in the chromosome for signal coverage
    for position in chromosome:
        # Check the signal strength at each position
        signals = data[data['position'] == position]['signal_strength']
        if not signals.empty:
            signal_strength = signals.values[0]
            # Count if within the desired signal range
            if min_signal <= signal_strength <= max_signal:
                covered_points += 1

    # Calculate coverage as a percentage of total points
    coverage = covered_points / total_points
    score = coverage * 100  # Convert coverage to percentage
    
    # Apply penalty for number of routers used (to encourage fewer routers if possible)
    penalty = len(chromosome) * 2  # Higher penalty factor encourages minimizing routers
    final_score = score - penalty
    
    # Adjust the score for cases that do not meet the coverage threshold
    if coverage < coverage_threshold:
        final_score -= 100  # Apply large penalty for low-coverage solutions
    
    return final_score


# Genetic algorithm with higher mutation rate
def genetic_algorithm_with_high_mutation(data, population_size=50, initial_routers=1, max_routers=10, generations=100, mutation_rate=0.2, coverage_threshold=0.75, layout_center=(0, 0), max_distance_from_center=10):
    num_routers = initial_routers
    best_placement = None
    best_score = -float('inf')
    
    while num_routers <= max_routers:
        # Initialize population
        population = initialize_population(data, population_size, num_routers, layout_center, max_distance_from_center)
        
        for generation in range(generations):
            # Calculate fitness scores using the high mutation fitness function
            fitness_scores = [fitness_function_high_mutation(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
            # Select parents for next generation
            population = selection(population, fitness_scores)
            
            next_generation = []
            for i in range(0, len(population), 2):
                parent1, parent2 = population[i], population[(i+1) % len(population)]
                child1, child2 = crossover(parent1, parent2)
                next_generation.extend([child1, child2])
            
            # Use a higher mutation rate
            low_signal_positions = data[data['signal_strength'] < -70]['position'].tolist()
            next_generation = [mutation(child, low_signal_positions, mutation_rate) for child in next_generation]
            
            # Update population
            population = next_generation
        
        # Evaluate final population fitness
        fitness_scores = [fitness_function_high_mutation(chromosome, data, coverage_threshold=coverage_threshold) for chromosome in population]
        current_best_score = max(fitness_scores)
        current_best_chromosome = population[fitness_scores.index(current_best_score)]
        
        # Update best placement if a better score is found
        if current_best_score > best_score:
            best_score = current_best_score
            best_placement = current_best_chromosome
            
            if fitness_function_high_mutation(best_placement, data, coverage_threshold=coverage_threshold) >= coverage_threshold * 100:
                break
            
        num_routers += 1

    return best_placement

# Selection (tournament selection)
def selection(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover with check for single-element chromosomes
def crossover(parent1, parent2):
    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Skip crossover if not enough points
    
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation (randomly swap router positions)
def mutation(chromosome, low_signal_positions, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(low_signal_positions)
    return chromosome

def main():
    # Load data
    data = load_signal_data('signal_degradation.txt')
    
    # Calculate layout center based on signal data
    layout_center_x = (data['position'].apply(lambda p: p[0]).max() + data['position'].apply(lambda p: p[0]).min()) // 2
    layout_center_y = (data['position'].apply(lambda p: p[1]).max() + data['position'].apply(lambda p: p[1]).min()) // 2
    layout_center = (layout_center_x, layout_center_y)

    # Genetic algorithm parameters
    population_size = 50
    initial_routers = 1
    max_routers = 4
    generations = 100
    mutation_rate = 0.1
    coverage_threshold = 0.75
    max_distance_from_center = 15  # Adjust as needed

    best_placement = genetic_algorithm_with_high_mutation(
        data, 
        population_size, 
        initial_routers, 
        max_routers, 
        generations, 
        mutation_rate, 
        coverage_threshold,
        layout_center,
        max_distance_from_center
    )
    
    # Calculate fitness score for best placement
    best_score = fitness_function_high_mutation(best_placement, data, coverage_threshold=coverage_threshold)
    
    # Format the output as specified
    num_routers = len(best_placement)
    report_content = f"Optimal Router Positions: ({num_routers}, {best_placement})\nFitness Score: {best_score}\n"
    
    # Display in console
    print(report_content)
    
    # Save results to report.txt in specified format
    report_file_path = 'report.txt' 
    with open(report_file_path, 'w') as report_file:
        report_file.write(report_content)

if __name__ == "__main__":
    main()

Optimal Router Positions: (1, [(9, 3)])
Fitness Score: -102.0

